Demonstrate a frequency scan of a simulated resonance.

This follows the steps in ../zcu216/mkids_2x2_kidsim_v2.ipynb, but should work in general on all platforms.
The parameter "bitfile" is set to select the firmware used, and it should work with all available firmwares.
Note that the previous demo_00_init.ipynb show how to find available firmwares on the platform.


For this notebook, we use the "Dual 0" chain for mkids software.  It uses these connections:
* DAC output on DAC Tile = 2, DAC Ch = 0, which is the connector 0_230, on JHC3
* ADC input on ADC Tile = 2, ADC Ch = 0, which is the connector 0_226, on JHC7

We use the "Sim 0" chain to simulate the resonance.  It uses these connections:
* DAC output on DAC Tile = 3, DAC Ch = 0, which is the connector 0_231, on JHC3
* ADC input on ADC Tile = 3, ADC Ch = 0, which is the connector 0_227, on JHC7

These are connected through the low-frequency Baluns.


In [ ]:
import sys
sys.path.append('../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
# After running demo_00_init.ipynb we decide which bitfile we'd like to use
bitfile = "mkids_2x2_kidsim_v2"
scan = Scan.Scan(bitfile)

In [ ]:
# Confirm what we have available
print(scan.soc)


In [ ]:
# Build processing chain
chain = KidsChain(scan.soc, dual=scan.soc['dual'][0])



In [ ]:
# Build simulation chain.
simu = SimuChain(scan.soc, simu=scan.soc['simu'][0])

# Set quantization.
simu.analysis.qout(3)
simu.synthesis.qout(3)

# Disable all resonators.
simu.alloff()

# Channel separation.
fc = simu.analysis.fc_ch

# Set mixer.
fMixerSimu = 300
simu.set_mixer_frequency(fMixerSimu)

# Pue one resonator 
fr0 = fMixerSimu + 10*fc
print("fr0 = {} MHz".format(fr0))

simu.enable(fr0+0.1)


In [ ]:
#######################
### Frequency Sweep ###
#######################
chain.analysis.qout(2)
chain.synthesis.qout(2)
fc = 324
df = 20
f,a,phi=chain.sweep(fc-df/2,fc+df/2,N=500,g=0.1)

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");